In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

In [2]:
NFOLDS = 3
SEED = 0
NROWS = None

data = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')
prev = pd.read_csv('previous_application.csv')

In [3]:
categorical_feats = [f for f in data.columns if data[f].dtype == 'object']

for i in categorical_feats:
    data[i], indexer = pd.factorize(data[i])
    test[i] = indexer.get_indexer(test[i])
    
gc.enable()

y_train = data['TARGET']
del data['TARGET']

prev_cat_features = [i for i in prev.columns if prev[i].dtype == 'object']

for i in prev_cat_features:
    prev[i], _ = pd.factorize(prev[i])

In [4]:
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['np_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [10]:
x_train = data.merge(right = avg_prev.reset_index(), how = 'left', on = 'SK_ID_CURR')
x_test = test.merge(right = avg_prev.reset_index(), how = 'left', on = 'SK_ID_CURR')

x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

excluded_feats = ['SK_ID_CURR']
features = [i for i in x_train.columns if i not in excluded_feats]

x_train = x_train[features]
x_test = x_test[features]

kf = KFold(n_splits = NFOLDS, shuffle = True, random_state = SEED)

In [11]:
class SklearnWrapper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['random_state'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]
    
class CatboostWrapper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['random_state'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]
    
class LightGBMWrapper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['feature_fraction'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]
    
class XgbWrapper(object):
    def __init__(self, seed = 0, params = None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)
        
    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label = y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)
        
    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))
    
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]
        
        clf.train(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
        
    oof_test[:] = oof_test_skf.mean(axis = 0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

et_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'allow_writing_files': False
}

lightgbm_params = {
    'n_estimators':200,
    'learning_rate':0.1,
    'num_leaves':123,
    'colsample_bytree':0.8,
    'subsample':0.9,
    'max_depth':15,
    'reg_alpha':0.1,
    'reg_lambda':0.1,
    'min_split_gain':0.01,
    'min_child_weight':2    
}

In [12]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf= CatBoostClassifier, seed = SEED, params=catboost_params)
lg = LightGBMWrapper(clf = LGBMClassifier, seed = SEED, params = lightgbm_params)

xg_oof_train, xg_oof_test = get_oof(xg)
et_oof_train, et_oof_test = get_oof(et)
rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)

print("XG-CV: {}".format(sqrt(mean_squared_error(y_train, xg_oof_train))))
print("ET-CV: {}".format(sqrt(mean_squared_error(y_train, et_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, rf_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, cb_oof_train))))

x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

print("{},{}".format(x_train.shape, x_test.shape))

logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train)

test['TARGET'] = logistic_regression.predict_proba(x_test)[:,1]

0:	total: 136ms	remaining: 27.1s
1:	total: 179ms	remaining: 17.7s
2:	total: 214ms	remaining: 14s
3:	total: 253ms	remaining: 12.4s
4:	total: 292ms	remaining: 11.4s
5:	total: 329ms	remaining: 10.7s
6:	total: 381ms	remaining: 10.5s
7:	total: 415ms	remaining: 9.97s
8:	total: 457ms	remaining: 9.7s
9:	total: 499ms	remaining: 9.49s
10:	total: 539ms	remaining: 9.26s
11:	total: 581ms	remaining: 9.1s
12:	total: 629ms	remaining: 9.04s
13:	total: 669ms	remaining: 8.89s
14:	total: 710ms	remaining: 8.75s
15:	total: 750ms	remaining: 8.62s
16:	total: 782ms	remaining: 8.42s
17:	total: 821ms	remaining: 8.3s
18:	total: 856ms	remaining: 8.15s
19:	total: 891ms	remaining: 8.02s
20:	total: 928ms	remaining: 7.91s
21:	total: 972ms	remaining: 7.86s
22:	total: 1.02s	remaining: 7.83s
23:	total: 1.06s	remaining: 7.78s
24:	total: 1.1s	remaining: 7.68s
25:	total: 1.14s	remaining: 7.6s
26:	total: 1.17s	remaining: 7.52s
27:	total: 1.21s	remaining: 7.46s
28:	total: 1.25s	remaining: 7.37s
29:	total: 1.29s	remaining: 7.3

44:	total: 1.82s	remaining: 6.26s
45:	total: 1.86s	remaining: 6.22s
46:	total: 1.9s	remaining: 6.19s
47:	total: 1.95s	remaining: 6.17s
48:	total: 1.99s	remaining: 6.12s
49:	total: 2.02s	remaining: 6.07s
50:	total: 2.06s	remaining: 6.03s
51:	total: 2.1s	remaining: 5.98s
52:	total: 2.14s	remaining: 5.93s
53:	total: 2.17s	remaining: 5.88s
54:	total: 2.22s	remaining: 5.85s
55:	total: 2.26s	remaining: 5.81s
56:	total: 2.3s	remaining: 5.77s
57:	total: 2.34s	remaining: 5.72s
58:	total: 2.38s	remaining: 5.69s
59:	total: 2.42s	remaining: 5.64s
60:	total: 2.46s	remaining: 5.61s
61:	total: 2.5s	remaining: 5.56s
62:	total: 2.53s	remaining: 5.51s
63:	total: 2.57s	remaining: 5.46s
64:	total: 2.61s	remaining: 5.41s
65:	total: 2.64s	remaining: 5.37s
66:	total: 2.69s	remaining: 5.33s
67:	total: 2.72s	remaining: 5.29s
68:	total: 2.76s	remaining: 5.25s
69:	total: 2.8s	remaining: 5.21s
70:	total: 2.84s	remaining: 5.16s
71:	total: 2.88s	remaining: 5.12s
72:	total: 2.92s	remaining: 5.08s
73:	total: 2.96s	re

85:	total: 3.62s	remaining: 4.8s
86:	total: 3.66s	remaining: 4.75s
87:	total: 3.7s	remaining: 4.71s
88:	total: 3.74s	remaining: 4.66s
89:	total: 3.78s	remaining: 4.62s
90:	total: 3.82s	remaining: 4.58s
91:	total: 3.86s	remaining: 4.53s
92:	total: 3.89s	remaining: 4.48s
93:	total: 3.94s	remaining: 4.44s
94:	total: 3.98s	remaining: 4.39s
95:	total: 4.02s	remaining: 4.35s
96:	total: 4.06s	remaining: 4.31s
97:	total: 4.1s	remaining: 4.27s
98:	total: 4.14s	remaining: 4.22s
99:	total: 4.18s	remaining: 4.18s
100:	total: 4.22s	remaining: 4.13s
101:	total: 4.26s	remaining: 4.09s
102:	total: 4.3s	remaining: 4.05s
103:	total: 4.34s	remaining: 4.01s
104:	total: 4.38s	remaining: 3.96s
105:	total: 4.42s	remaining: 3.92s
106:	total: 4.46s	remaining: 3.87s
107:	total: 4.5s	remaining: 3.83s
108:	total: 4.54s	remaining: 3.79s
109:	total: 4.58s	remaining: 3.75s
110:	total: 4.62s	remaining: 3.7s
111:	total: 4.66s	remaining: 3.66s
112:	total: 4.71s	remaining: 3.62s
113:	total: 4.75s	remaining: 3.58s
114:	t

In [14]:
test['TARGET']

0        0.047240
1        0.110492
2        0.026113
3        0.038719
4        0.148671
           ...   
48739    0.042605
48740    0.075681
48741    0.027298
48742    0.064708
48743    0.221590
Name: TARGET, Length: 48744, dtype: float64